## Example workbook for DIC <-> EBSD linking

In [15]:
#Load third-party packages
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform as tf
from skimage import io

#Load quat, ebsd and hrdic packages from 'defdap' sub-directory
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic

#Set plot behaviour (grain selection has only been tested in osx display mode)
import os
if os.name == 'posix':
    %matplotlib osx
else:
    %matplotlib tk

### Load in and display DIC map

In [20]:
#Load in DIC map
DicFilePath = "example_data/"
DicMap = hrdic.Map(DicFilePath, "B00005.txt")

#Set crop
DicMap.setCrop(xMin=15, xMax=28, yMin=30, yMax=15)

Loaded DaVis 8.1.5 data (dimensions: 586 x 510 pixels, sub-window size: 12 x 12 pixels)


In [ ]:
#Display max shear map
DicMap.plotMaxShear(plotColourBar=True)

### Print stats table

In [3]:
DicMap.printStatsTable(percentiles=['Min', 5, 'Mean', 95,'Max'], components = ['mss', 'f11', 'f22'])

DicMap (dimensions: 586 x 510 pixels, sub-window size: 12 x 12 pixels, number of points: 298860)

Component	Min	P5	Mean	P95	Max	
mss		0.01	0.39	1.95	4.39	12.74	
f11		-9.26	-0.34	2.13	5.15	16.30	
f22		-13.78	-3.77	-1.10	0.88	7.63	



### Effective shear strain histogram

In [ ]:
data = [item for sublist in DicMap.crop(DicMap.max_shear) for item in sublist]

f, (ax1) = plt.subplots(1)
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

counts, bins, bars = ax1.hist(data, bins=np.linspace(0,0.2,100), color='r', histtype='step')
plt.gca().set_xlabel("Effective shear strain")
plt.gca().set_ylabel("Frequency")

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('irr.csv', (center, counts), delimiter='\n')

plt.show()

### Load in EBSD map then display local misorientation

In [22]:
#Load in EBSD map and calculate misorientation
EbsdFilePath = "example_data/Map Data 2-DIC area"
EbsdMap = ebsd.Map(EbsdFilePath, "cubic")
EbsdMap.loadSlipSystems(filepath="defdap/slip_systems/cubic.txt")

EbsdMap.binData = EbsdMap.binData[::-1]    #Rotate the map 180 degrees
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 6)       #Find boundaries
EbsdMap.findGrains(minGrainSize = 10)      #Find grains
EbsdMap.calcGrainMisOri(calcAxis = True)

Loaded EBSD data (dimensions: 1006 x 996 pixels, step size: 0.1 um)


In [ ]:
EbsdMap.plotMisOriMap(plotGBs=True, vmin=0, vmax=4)

### Find a grain of interest (click around the map)

In [ ]:
EbsdMap.locateGrainID()

### Plot misorientation for isolated grain

In [ ]:
EbsdMap.grainList[EbsdMap.currGrainId].plotMisOri()

In [ ]:
EbsdMap.grainList[EbsdMap.currGrainId].plotMisOri(component=4)

### Plot phase map

In [24]:
EbsdMap.plotPhaseMap()

## Link EBSD map to DIC map

### Define homologous points

In [4]:
#Set homologous points   (x, y)
DicMap.homogPoints = np.array((
    (303, 408), (120, 392), (275, 25)
))

EbsdMap.homogPoints = np.array((
    (536, 776), (236, 790), (466, 160)
))

### Display map with homologous points

In [ ]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

### Link the maps with a shift for corrections

In [5]:
DicMap.linkEbsdMap(EbsdMap)

### Plot max shear with boundaries

In [6]:
DicMap.plotMaxShear(plotGBs=True)

### Detect grains in the DIC map

In [ ]:
DicMap.findGrains(minGrainSize=10)

### Locate a grain of interest (click around)

In [ ]:
DicMap.locateGrainID(displaySelected=True)
#displaySelected will show the selcted grain in a separate figure window. 
#Do not close the window and it will update as you click grains.

### Plot max shear for isolated grain

In [ ]:
DicMap.grainList[DicMap.currGrainId].plotMaxShear()

### Plot a histogram of it

In [ ]:
plt.figure()
plt.hist(DicMap.grainList[DicMap.currGrainId].maxShearList, bins=100);

### Plot misorientation for isolated grain

In [ ]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
EbsdMap.grainList[ebsdGrainId].plotMisOri()

### Plot max shear for isolated grain with slip traces

In [ ]:
DicMap.grainList[DicMap.currGrainId].calcSlipTraces()
DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

### Plot boundaries without crop to see rotation

In [ ]:
plt.figure()

warpedBoundaries = tf.warp(-DicMap.ebsdMap.boundaries.astype(float), DicMap.ebsdTransform) > 0.1

warpedBoundaries = -warpedBoundaries.astype(int)

plt.imshow(warpedBoundaries, cmap="gray")